In [1]:
import cv2
import os
import time
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each person, enter one numeric face id
face_id = input('\n Make sure the first user entered is 0.enter user id end press <return> ==>  ')
face_name = input(f'\n Give the name for {face_id} <return> ==>  ')

print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0

file_path = 'dataset/labels.txt'

if not os.path.exists('dataset'):
    os.mkdir('dataset')


# Check if the file exists
file_exists = os.path.exists(file_path)

# Open the file in append mode (creates the file if it doesn't exist)
with open(file_path, 'a') as file:
    # Add a newline character if the file is new
    if file_exists:
        file.write('\n')
    
    # Append text to the file
    text_to_append = face_name
    file.write(text_to_append)

# Variables for FPS calculation
frame_count = 0
start_time = time.time()
fps = 0  # Initialize fps variable

while(True):

    ret, img = cam.read()
    img = cv2.flip(img, 1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w+50,y+h+50), (255,0,0), 2)     

        # Save the captured image into the datasets folder
        gray = gray[y:y+h,x:x+w]

        try:
            cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg",gray )
        except Exception as e:
            print(e)
            print(gray)
            continue


        # Calculate FPS and display it on the image
        frame_count += 1
        if frame_count >= 1:  # Calculate FPS every 30 frames (adjust as needed)
            end_time = time.time()
            elapsed_time = end_time - start_time
            fps = frame_count / elapsed_time
            frame_count = 0
            start_time = time.time()

        #Add FPS counter
        cv2.putText(img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        #Add number of images of face captures
        text_position = (img.shape[1] - 300, 30)
        cv2.putText(img, f"Img Captured: {count}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('image', img)
        count += 1


    k = cv2.waitKey(20) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 1000: # Take 1000 face sample and stop video
         break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 Make sure the first user entered is 0.enter user id end press <return> ==>  1

 Give the name for 1 <return> ==>  rahul

 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import numpy as np
import cv2
import os
import h5py
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras import backend as K 
from sklearn.model_selection import train_test_split
from Model import model
from keras import callbacks

# Path for face image database
path = 'dataset'

detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");


def downsample_image(img):
    img = Image.fromarray(img.astype('uint8'), 'L')
    img = img.resize((32,32), Image.LANCZOS)
    return np.array(img)



# function to get the images and label data
def getImagesAndLabels(path):
    
    path = 'dataset'
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:
        
        #if there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        except:
            continue    
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)
    return faceSamples,ids

print ("\n [INFO] Training faces now.")
faces,ids = getImagesAndLabels(path)

K.clear_session()
n_faces = len(set(ids))
model = model((32,32,1),n_faces)
# faces = np.asarray(faces)
faces = np.array([downsample_image(ab) for ab in faces])
ids = np.asarray(ids)
faces = faces[:,:,:,np.newaxis]
print("Shape of Data: " + str(faces.shape))
print("Number of unique faces : " + str(n_faces))


ids = to_categorical(ids)

faces = faces.astype('float32')
faces /= 255.





 [INFO] Training faces now.



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization (Batch  (None, 28, 28, 64)        256       
 Normalization)                                                  
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        4160      
                       

In [3]:
x_train, x_test, y_train, y_test = train_test_split(faces,ids, test_size = 0.2, random_state = 0)

In [4]:
checkpoint = callbacks.ModelCheckpoint('./trained_model.h5', monitor='val_accuracy',
                                           save_best_only=True, save_weights_only=True, verbose=1)
model.fit(x_train, y_train,
             batch_size=32,
             epochs=100,
             validation_data=(x_test, y_test),
             shuffle=True,callbacks=[checkpoint])

# Print the numer of faces trained and end program
print("\n [INFO] " + str(n_faces) + " faces trained. Exiting Program")

Epoch 1/10


50/50 [==============================] - ETA: 0s - loss: 0.6815 - accuracy: 0.8712
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to .\trained_model.h5
50/50 [==============================] - 8s 141ms/step - loss: 0.6815 - accuracy: 0.8712 - val_loss: 0.7190 - val_accuracy: 0.5000
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.0901 - accuracy: 0.9669
Epoch 2: val_accuracy did not improve from 0.50000
50/50 [==============================] - 7s 135ms/step - loss: 0.0901 - accuracy: 0.9669 - val_loss: 0.8807 - val_accuracy: 0.5000
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.0389 - accuracy: 0.9931
Epoch 3: val_accuracy did not improve from 0.50000
50/50 [==============================] - 6s 124ms/step - loss: 0.0389 - accuracy: 0.9931 - val_loss: 1.0543 - val_accuracy: 0.5000
Epoch 4/10
50/50 [==============================] - ETA: 0s - loss: 0.0234 - accuracy: 0.9956
Epoch 4: val_accuracy did not improve f

In [13]:
import numpy as np
import cv2
import os
import h5py
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.layers import Dense, Activation, Flatten
from PIL import Image
from Model import model
import time


def getImagesAndLabels():
    path = 'dataset'
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:

        #if there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        except:
            continue  
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)
    return faceSamples,ids
    

_,ids = getImagesAndLabels()
model = model((32,32,1),len(set(ids)))
model.load_weights  ('trained_model.h5')
model.summary()

# Open the file in read mode
with open('Dataset/labels.txt', 'r') as file:
    # Read the entire file content
#     labels = file.read().split('\t')
    labels = file.readlines()
    


cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
font = cv2.FONT_HERSHEY_SIMPLEX
def start():

    # Variables for FPS calculation
    frame_count = 0
    start_time = time.time()
    fps = 0  # Initialize fps variable

    cap = cv2.VideoCapture(0)
    print('here')
    ret = True

    clip = []
    while ret:
        #read frame by frame
        ret, frame = cap.read()
        nframe = frame
        faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))


        # Calculate FPS and display it on the image
        frame_count += 1
        if frame_count >= 1:  # Calculate FPS every 30 frames (adjust as needed)
            end_time = time.time()
            elapsed_time = end_time - start_time
            fps = frame_count / elapsed_time
            print(frame_count, ' /q ' , elapsed_time)
            frame_count = 0
            start_time = time.time()

        #Add FPS counter
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
        try:
            (x,y,w,h) = faces[0]
        except:
            continue
        frame = frame[y:y+h,x:x+w]
        frame = cv2.resize(frame, (32,32))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('result small' , frame)
        c= cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
        
#         gray = gray[np.newaxis,:,:,np.newaxis]
        gray = gray.reshape(-1, 32, 32, 1).astype('float32') / 255.
        print(gray.shape)
        prediction = model.predict(gray)
        print("prediction:" + str(prediction))

        print("\n\n\n\n")
        print("----------------------------------------------")
        # labels = ['Vivek']
        prediction = prediction.tolist()
        
        listv = prediction[0]
        n = listv.index(max(listv))
        print("\n")
        print("----------------------------------------------")
#         print( "Highest Probability: " + "User " + str(n) + "==>" + str(prediction[0][n]) )
        print( "Highest Probability: " + labels[n] + "==>" + str(round(prediction[0][n],3)) )
        
        print("----------------------------------------------")
        print("\n")
        for (x, y, w, h) in faces:
            try:
                cv2.rectangle(nframe, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(nframe, str(labels[n]), (x+5,y-5), font, 1, (255,255,255), 2)
#                 cv2.putText(nframe, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
            except:
                la = 2 
        prediction = np.argmax(model.predict(gray), 1)
        print(prediction)
        cv2.imshow('result', nframe)

        c = cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
start()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 activation_54 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 conv2d_46 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_18 (Ba  (None, 28, 28, 64)        256       
 tchNormalization)                                               
                                                                 
 activation_55 (Activation)  (None, 28, 28, 64)        0         
                                                                 
 conv2d_47 (Conv2D)          (None, 28, 28, 64)        4160      
                                                      

1/1 [==============================] - 0s 24ms/step
prediction:[[0.82299846 0.17700154]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.823
----------------------------------------------


1/1 [==============================] - 0s 28ms/step
[0]
1  /q  0.1799764633178711
(1, 32, 32, 1)
1/1 [==============================] - 0s 24ms/step
prediction:[[0.861785   0.13821505]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.862
----------------------------------------------


1/1 [==============================] - 0s 24ms/step
[0]
1  /q  0.18482637405395508
(1, 32, 32, 1)
1/1 [==============================] - 0s 24ms/step
prediction:[[0.86158323 0.13841678]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.862
-----------------------------

1/1 [==============================] - 0s 29ms/step
prediction:[[0.92470616 0.07529378]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.925
----------------------------------------------


1/1 [==============================] - 0s 27ms/step
[0]
1  /q  0.17409825325012207
(1, 32, 32, 1)
1/1 [==============================] - 0s 30ms/step
prediction:[[0.9258886  0.07411143]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.926
----------------------------------------------


1/1 [==============================] - 0s 27ms/step
[0]
1  /q  0.20545339584350586
(1, 32, 32, 1)
1/1 [==============================] - 0s 22ms/step
prediction:[[0.91630936 0.08369067]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.916
----------------------------

1/1 [==============================] - 0s 26ms/step
[0]
1  /q  0.1847846508026123
1  /q  0.03208017349243164
1  /q  0.028403043746948242
1  /q  0.024501800537109375
1  /q  0.0335230827331543
1  /q  0.02553558349609375
1  /q  0.02687525749206543
1  /q  0.025259971618652344
1  /q  0.022275924682617188
1  /q  0.03748321533203125
1  /q  0.0384063720703125
1  /q  0.02724146842956543
1  /q  0.025048494338989258
1  /q  0.028714418411254883
1  /q  0.02727961540222168
1  /q  0.024823904037475586
1  /q  0.058710336685180664
1  /q  0.03202247619628906
1  /q  0.03293347358703613
1  /q  0.02369236946105957
1  /q  0.03784608840942383
1  /q  0.02199554443359375
1  /q  0.060289859771728516
1  /q  0.025340557098388672
1  /q  0.03147125244140625
1  /q  0.025730133056640625
1  /q  0.020954608917236328
1  /q  0.024547100067138672
1  /q  0.050066232681274414
1  /q  0.026109933853149414
1  /q  0.051401376724243164
1  /q  0.015158414840698242
1  /q  0.05016016960144043
1  /q  0.02929544448852539
1  /q  0.041

1/1 [==============================] - 0s 26ms/step
prediction:[[0.7824261  0.21757393]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.782
----------------------------------------------


1/1 [==============================] - 0s 26ms/step
[0]
1  /q  0.21371984481811523
1  /q  0.033565521240234375
(1, 32, 32, 1)
1/1 [==============================] - 0s 29ms/step
prediction:[[0.73049396 0.26950598]]





----------------------------------------------


----------------------------------------------
Highest Probability: vivek
==>0.73
----------------------------------------------


1/1 [==============================] - 0s 38ms/step
[0]
1  /q  0.1995527744293213
1  /q  0.017127037048339844
1  /q  0.0334780216217041
1  /q  0.034096717834472656
1  /q  0.040255069732666016
1  /q  0.027133941650390625
1  /q  0.05180215835571289
1  /q  0.027009963989257812
1  /q  0.022280454635620117
1  /q  0.0316841602325

1  /q  0.040331125259399414
1  /q  0.03786754608154297
1  /q  0.027542591094970703
1  /q  0.0416719913482666
1  /q  0.04094529151916504
1  /q  0.03342294692993164
1  /q  0.03777360916137695
1  /q  0.029271364212036133
1  /q  0.04206562042236328
1  /q  0.050687551498413086
1  /q  0.032713890075683594
1  /q  0.036463022232055664
1  /q  0.034455060958862305
1  /q  0.034595489501953125
1  /q  0.03280973434448242
1  /q  0.033447265625
1  /q  0.033968448638916016
1  /q  0.02696847915649414
1  /q  0.040535926818847656
1  /q  0.035012245178222656
1  /q  0.02468132972717285
1  /q  0.027189016342163086
1  /q  0.04477381706237793
1  /q  0.03310990333557129
1  /q  0.038224220275878906
1  /q  0.037891387939453125
1  /q  0.033718109130859375
1  /q  0.03444337844848633
1  /q  0.033492326736450195
1  /q  0.03839254379272461
1  /q  0.03878593444824219
1  /q  0.033447265625
1  /q  0.023468494415283203
1  /q  0.04487752914428711
1  /q  0.0346527099609375
1  /q  0.03474593162536621
1  /q  0.03941154479980